In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os
#run from repo root...
os.chdir(r"C:/Users/jamed\Documents/repos/lidar_2/LidarClassifcationForFloodModelling")

In [15]:
from structured_data_utils.structuring import GeotiffGeneration, get_combined_geotiff_tensor

In [16]:
GeotiffGeneration.generate_geotiffs()

In [17]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_tensor(tensor):
    image_np = tensor.detach().cpu().numpy().astype(np.float32)

    if image_np.ndim == 3 and image_np.shape[0] <= 4:
        image_np = np.moveaxis(image_np, 0, -1)  # CHW → HWC

    if image_np.ndim == 2:
        image_np = image_np[:, :, None]

    if image_np.shape[2] > 3:
        image_np = image_np[:, :, :3]

    vmin = np.nanmin(image_np, axis=(0, 1), keepdims=True)
    vmax = np.nanmax(image_np, axis=(0, 1), keepdims=True)

    image_np = (image_np - vmin) / (vmax - vmin + 1e-8)

    plt.imshow(image_np.squeeze())
    plt.axis("off")
    plt.show()


#visualize_tensor(combined_tensor)

In [18]:
from structured_data_utils.data import standardise_core_geotiffs
standardise_core_geotiffs()

CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:7856', '-tr', '0.5', '0.5', '-r', 'bilinear', '-overwrite', 'data/POSITIVE.tif', 'data/POSITIVE_STANDARDISED.tif'], returncode=0, stdout='Creating output file that is 3358P x 4940L.\nUsing internal nodata values (e.g. -9999) for image data/POSITIVE.tif.\nCopying nodata values from source data/POSITIVE.tif to destination data/POSITIVE_STANDARDISED.tif.\nProcessing data/POSITIVE.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n', stderr='')
CompletedProcess(args=['gdalwarp', '-t_srs', 'EPSG:7856', '-tr', '0.5', '0.5', '-r', 'bilinear', '-overwrite', 'data/NEGATIVE.tif', 'data/NEGATIVE_STANDARDISED.tif'], returncode=0, stdout='Creating output file that is 4000P x 6000L.\nUsing internal nodata values (e.g. -9999) for image data/NEGATIVE.tif.\nCopying nodata values from source data/NEGATIVE.tif to destination data/NEGATIVE_STANDARDISED.tif.\nProcessing data/NEGATIVE.tif [1/1] : 0...10...20...30...40...50...60...70..

In [19]:
combined_tensor = get_combined_geotiff_tensor()

In [20]:
from structured_data_utils.structured_data_interfacing import get_segments_with_sliding_window

In [21]:
combined_tensor[2][1]

tensor(33.1353)

In [22]:
segments = get_segments_with_sliding_window(combined_tensor)

14000
range(0, 14000, 25)


In [23]:
segments

[tensor([[33.1543, 33.1524, 33.1487,  ...,     nan,     nan,     nan],
         [33.1496, 33.1467, 33.1409,  ...,     nan,     nan,     nan],
         [33.1402, 33.1353, 33.1253,  ...,     nan,     nan,     nan],
         ...,
         [33.6100, 33.6037, 33.5912,  ...,     nan,     nan,     nan],
         [33.6240, 33.6193, 33.6101,  ...,     nan,     nan,     nan],
         [33.6391, 33.6354, 33.6281,  ...,     nan,     nan,     nan]]),
 tensor([[31.9692, 31.9416, 31.8863,  ...,     nan,     nan,     nan],
         [31.9754, 31.9527, 31.9072,  ...,     nan,     nan,     nan],
         [32.1078, 32.0897, 32.0534,  ...,     nan,     nan,     nan],
         ...,
         [34.0911, 34.0855, 34.0743,  ...,     nan,     nan,     nan],
         [34.1025, 34.0970, 34.0859,  ...,     nan,     nan,     nan],
         [34.1138, 34.1082, 34.0969,  ...,     nan,     nan,     nan]]),
 tensor([[33.6554, 33.6520, 33.6451,  ...,     nan,     nan,     nan],
         [33.6751, 33.6710, 33.6628,  ...,   

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def visualize_segments(segments, title=None, show_colorbar=True, pad_value=float("-inf")):
    """
    Handles variable-sized 2D segment tensors by padding them to a common (H, W),
    then rendering a categorical map where each pixel is assigned to the segment
    with the highest value at that pixel.

    segments: list/tuple of 2D tensors/arrays, OR a 3D tensor (C,H,W)
    pad_value: value used to pad missing areas; -inf ensures padded areas never win
    """
    # Normalize input to list of 2D torch tensors
    if isinstance(segments, torch.Tensor):
        if segments.ndim != 3:
            raise ValueError(f"Expected 3D tensor (C,H,W), got {segments.ndim}D")
        stack = segments.to(dtype=torch.float32)
        C, H, W = stack.shape
        finite = torch.isfinite(stack)
        valid_mask = finite.any(dim=0)
        stack_clean = torch.where(finite, stack, torch.tensor(float("-inf"), device=stack.device))
        labels = torch.argmax(stack_clean, dim=0)
        labels_np = labels.detach().cpu().numpy().astype(np.int32)
        valid_np = valid_mask.detach().cpu().numpy()
        labels_np = np.where(valid_np, labels_np, -1)
        C = stack.shape[0]
    else:
        if not isinstance(segments, (list, tuple)) or len(segments) == 0:
            raise ValueError("segments must be a non-empty list/tuple of 2D tensors/arrays, or a 3D tensor.")

        seg_tensors = []
        shapes = []
        device = None

        for i, s in enumerate(segments):
            t = s if isinstance(s, torch.Tensor) else torch.as_tensor(s)
            if t.ndim != 2:
                raise ValueError(f"Each segment must be 2D (H,W). Got shape {tuple(t.shape)}")
            if device is None:
                device = t.device
            else:
                # keep everything on the same device
                t = t.to(device)
            t = t.to(dtype=torch.float32)
            seg_tensors.append(t)
            shapes.append(t.shape)
            if i>10:
                break

        # Target size = max H, max W
        H = max(h for h, w in shapes)
        W = max(w for h, w in shapes)
        C = len(seg_tensors)

        # Build padded stack (C,H,W)
        stack = torch.full((C, H, W), pad_value, dtype=torch.float32, device=device)

        for i, t in enumerate(seg_tensors):
            h, w = t.shape
            # keep NaNs as -inf so they don't win argmax
            t_clean = torch.where(torch.isfinite(t), t, torch.tensor(float("-inf"), device=device))
            stack[i, :h, :w] = t_clean

        # Valid where at least one segment has finite value
        valid_mask = torch.isfinite(stack).any(dim=0)

        labels = torch.argmax(stack, dim=0)  # (H,W)
        labels_np = labels.detach().cpu().numpy().astype(np.int32)
        valid_np = valid_mask.detach().cpu().numpy()
        labels_np = np.where(valid_np, labels_np, -1)

    # Plot
    cmap = plt.cm.get_cmap("tab20", C)
    cmap.set_bad(alpha=0.0)

    fig, ax = plt.subplots()
    im = ax.imshow(np.ma.masked_where(labels_np < 0, labels_np), cmap=cmap, vmin=0, vmax=C-1)
    ax.axis("off")
    if title:
        ax.set_title(title)

    if show_colorbar:
        cbar = plt.colorbar(im, ax=ax, ticks=np.arange(C))
        cbar.set_label("Segment index")
        cbar.ax.set_yticklabels([str(i) for i in range(C)])

    plt.show()

# Example usage:
visualize_segments(segments, title="Segments (argmax assignment)")


ValueError: Each segment must be 2D (H,W). Got shape ()